<a href="https://colab.research.google.com/github/EdoardoMorucci/Plant-Leaves-Search-Engine---MIRCV/blob/main/Performance_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
def compute_ap_at_k(ranks, groundtruth, k):
  """ Computes the Average Precision @ k.
  Args:
    - ranks (ndarray): a (Q,N)-shaped matrix of indices of retrieved elements
                       for each query, already ranked by similarity. E.g.,
                       ranks[0, 4] is the index of fifth retrieved element with
                       respect to query #0.
    - groundtruth (ndarray): a (Q,N)-shaped boolean matrix indicating relevance
                             of database elements to queries, i.e., if
                             groundtruth[3,2] is True, then the database element
                             #2 is relevant to the query #3.
    - k (int): the number of top retrieved elements to be considered.

  Returns:
    - aps (ndarray): a (Q,)-shaped array containing the Average Precision @ K
                     for each query.
  """
  n_queries = len(ranks)
  # get relevance of retrieved elements
  ranked_relevance = groundtruth[np.arange(n_queries).reshape(-1, 1), ranks]

  total_relevant_items = ranked_relevance.sum(axis=1)

  # Precision@i for each query, i.e. the number of relevant items @i divided by i
  prec_at_i = ranked_relevance[:, :k].cumsum(axis=1) / np.arange(1, k + 1) 

  # sum of prec_at_i only for relevant, divide for total number of relevant items (for each query)
  aps = (ranked_relevance[:, :k] * prec_at_i).sum(axis=1) / total_relevant_items

  return aps

In [ ]:
def display_results(ranks, groundtruth, queries_annot, db_annot, limit=10):
  """ Draws results of image searches.
  Args:
    - ranks (ndarray): a (Q,N)-shaped matrix of indices of retrieved elements
                       for each query, already ranked. E.g., ranks[0, 4] is the
                       index of fifth retrieved element with respect to query #0.
    - groundtruth (ndarray): a (Q,N)-shaped boolean matrix indicating relevance
                             of database elements to queries, i.e., if
                             groundtruth[3,2] is True, then the database element
                             #2 is relevant to the query #3.
    - queries_annot (pandas.DataFrame): information about queries.
    - db_annot (pandas.DataFrame): information about database elements.
    - limit (int): the number of top results to show.
  """
  nq = len(ranks)

  image_paths = []
  image_relevance = []
  for i in range(nq):
    image_paths.append( queries_annot.iloc[i].ImagePath )
    image_relevance.append( None )

    image_paths.extend( db_annot.iloc[ranks[i, :limit]].ImagePath.tolist() )
    image_relevance.extend( groundtruth[i, ranks[i, :limit]].tolist() )

  grid = make_image_grid(image_paths, image_relevance, rows=nq, cols=limit+1)
  display(grid)

In [5]:
import tensorflow as tf
from tensorflow import keras

In [6]:
model_raw = tf.keras.applications.densenet.DenseNet121(
    weights='imagenet', input_tensor=None, input_shape=(224,224,3), 
    pooling='avg', include_top=False
)

model_finetuned= keras.models.load_model('/content/gdrive/Shareddrives/MIRCV-PlantLeavesSearchEngine/model_fine_tuned')

29097984/29084464 [==============================] - 0s 0us/step


In [8]:
! unzip /content/gdrive/Shareddrives/MIRCV-PlantLeavesSearchEngine/healthy-unhealthy-plants-sets.zip

Output streaming troncato alle ultime 5000 righe.
  inflating: content/healthy-unhealthy-plants-sets/test/Tomato/05138150-866c-460e-a252-19d269a81d4d___Crnl_L.Mold 6745_final_masked.jpg  
  inflating: content/healthy-unhealthy-plants-sets/test/Tomato/40761.jpg  
  inflating: content/healthy-unhealthy-plants-sets/test/Tomato/67206390-2511-488b-82a6-7e3c01b37987___YLCV_NREC 0071_final_masked.jpg  
  inflating: content/healthy-unhealthy-plants-sets/test/Tomato/17ea3930-bd36-4e02-8bef-91511c6a716e___Crnl_L.Mold 6862_final_masked.jpg  
  inflating: content/healthy-unhealthy-plants-sets/test/Tomato/e25c59a0-9362-4218-82de-9060174d5473___GCREC_Bact.Sp 6277_final_masked.jpg  
  inflating: content/healthy-unhealthy-plants-sets/test/Tomato/5300bf32-485d-4978-9099-3175fb666da8___GH_HL Leaf 475_final_masked.jpg  
  inflating: content/healthy-unhealthy-plants-sets/test/Tomato/4fc2334d-be0a-45f9-970d-be7a0f6a6c2a___RS_Late.B 6268_final_masked.jpg  
  inflating: content/healthy-unhealthy-plants-sets/

In [ ]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    percorso,
    labels='inferred', #the label of the dataset is obtained by the name of the directory
    seed=123,
    shuffle=False,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
)